# Task 3 
 > When we capture LiDAR data from a moving vehicle, each scan comes with its own local coordinate system, where the car/sensor is centered at x=0, y=0, z=0. However, aligning these scans into a unified world coordinate system brings significant benefits for visualization and downstream tasks that benefit from temporal information. In this context, 1) familiarize yourself with the concept of "ego-motion compensation" concerning LiDAR scans captured from a moving vehicle and 2) utilize the provided pose data from SemanticKITTI to create a function that compensates for the vehicle's ego-motion and aligns each scan into a shared world coordinate system. You can find anything you need for this purpose in this github repository: https://github.com/PRBonn/semantic-kitti-api , but its no free lunch, you have to search a bit and understand :). Finally, visualize a sequence of point clouds both with and without ego-motion compensation in two ways: A) Simply visualize 20 subsequent frames at the same time, B) visualize one frame at a time using your visualizer from Task 2. 
=> What difference do you observe when visualizing a point cloud sequence with and without ego-motion compensation? 

In this task we aim to visualize the LiDAR data from SemanticKITTI dataset with ego-motion compenstation. In the previous task, the coordinate system of the visualization was fixed at scanner i.e. (0,0,0) was defined to be the position of LiDAR scanner.<br> Now we want to have a global coordinate system for all sequences. 



## Theory

#### Coordinate Transformation
A point  $\mathbf{P}$ in 3d space can be represented  as a vector with 3 elements $\mathbf{p} = (p_x, p_y, p_z)$.
<br>Each element corresponds to its coordinate along one of the axes (x, y or z)
$$
\mathbf{P} =
\begin{bmatrix}
p_x \\
p_y \\
p_z
\end{bmatrix}
.
$$

To rotate a point in 3D space, is to multiply a rotation matrix with the position vector of a point. Formally, rotaion is a form of linar transformation as we morph(squeeze or stretch) the 3D space in a linear fashion.

$$
\mathbf{p}' = \mathbf{R}\mathbf{p}
$$
with $\mathbf{R}$ a \(3 $\times$ 3\) rotation matrix.

For coordinate transformation, rotation is not enough. We also need a component for translation
Thus coordinate transformations can be represented by 
$$
{}^{a}\mathbf{p} = {}^{a}\mathbf{r}_{b} + {}^{a}\mathbf{R}_{b} {}^{b}\mathbf{p}
$$

where <br>
$ ^{a}\mathbf{p}$ is the position vector of point $\mathbf{p}$ in coordinate system $\mathbf{S}_{a}$ <br>
$^{a}\mathbf{r}_{b}$ is the translation vector from coordinate system $\mathbf{S}_{a}$ to $\mathbf{S}_{b}$  <br>
$^{a}\mathbf{R}_{b}$ is the rotation matrix between $\mathbf{S}_{a}$ and  $\mathbf{S}_{b}$ <br>
$^{b}\mathbf{p}$ is the position vector of point $\mathbf{p}$ in coordinate system $\mathbf{S}_{b}$<br>


#### Homogenous Transformation
The above formula works just fine, but there is a more concise way to do this.
First we define something called a transformation matrix.
$$
{}^{a}\mathbf{T}_{b} = \begin{pmatrix}
{}^{a}\mathbf{R}_{b} & {}^{a}\mathbf{r}_{b} \\
\mathbf{0}^{T} & 1
\end{pmatrix}
$$

<br> ${}^{a}\mathbf{T}_{b} \in \mathbb{R}^{4 \times 4}$ is the transformation matrix from $\mathbf{S}_{a}$ to $\mathbf{S}_{b}$


We also redefine the position vector to 
$$
{}^{a}\hat{\mathbf{p}} = \begin{pmatrix}
{}^{a}\mathbf{p} \\
1
\end{pmatrix}
$$

such that ${}^{a}\hat{\mathbf{p}} \in \mathbb{R}^{4 \times 1}$ 

So the final equation for transformation from coordinate system $\mathbf{S}_{b}$ to $\mathbf{S}_{a}$ becomes

$$
{}^{a}\hat{\mathbf{p}} = {}^{a}\mathbf{T}_{b} \, {}^{b}\hat{\mathbf{p}}
$$



### Transformation of point clouds

Now we are given pose data for each sequence. This means we have the transformation matrix for each sequence. Thus all it remains is to multiply each point of a sequence with the transformation matrix and then visualize the entire scene after we are done commputing the new coordinates for each point


# Implementation

+ Use arrow keys "left" and "right" to go forward or backward by 1 sequence
+ Use 'n' to go forward 20 sequences
+ Use  'b' to go backward 20 sequences

In [1]:
import open3d as o3d
import numpy as np
import os
import json
import logging
from typing import Dict, List
from io import StringIO

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class SemanticKITTIVisualizer:
    """
    A class to visualize SemanticKITTI dataset using Open3D.

    Attributes:
        base_dir (str): Base directory of the dataset.
        seq_idx (str): Sequence index.
        color_map (dict): Dictionary mapping labels to colors.
        view_status_file (str): Path to the view status file.
        point_size (int): Point size for visualization.
        scan_dir (str): Directory containing the scan files.
        label_dir (str): Directory containing the label files.
        pose_file (str): Path to the poses file.
        scan_files (list): List of scan file names.
        label_files (list): List of label file names.
        pcd (o3d.geometry.PointCloud): Open3D point cloud object.
        geometry_added (bool): Flag to check if geometry is already added to the visualizer.
    """

    def __init__(self, base_dir: str, seq_idx: str, color_map_file: str, view_status_file: str, point_size: int) -> None:
        """
        Initializes the SemanticKITTIVisualizer class.

        Parameters:
            base_dir (str): Base directory of the dataset.
            seq_idx (str): Sequence index.
            color_map_file (str): Path to the color map file.
            view_status_file (str): Path to the view status file.
            point_size (int): Point size for visualization.
        """
        self.base_dir = base_dir
        self.seq_idx = seq_idx
        self.color_map = self.load_color_map(color_map_file)
        self.view_status_file = view_status_file
        self.point_size = point_size
        self.scan_dir = os.path.join(base_dir, seq_idx, "velodyne")
        self.label_dir = os.path.join(base_dir, seq_idx, "labels")
        self.pose_file = os.path.join(base_dir, seq_idx, "poses.txt")
        self.scan_files = sorted(os.listdir(self.scan_dir))
        self.label_files = sorted(os.listdir(self.label_dir))
        self.pcd = o3d.geometry.PointCloud()
        self.geometry_added = False
        logging.info("SemanticKITTIVisualizer initialized")

    def load_color_map(self, color_map_file: str) -> Dict[int, np.ndarray]:
        """
        Loads the color map from a file.

        Parameters:
            color_map_file (str): Path to the color map file.

        Returns:
            Dict[int, np.ndarray]: A dictionary mapping labels to colors.
        """
        with open(color_map_file) as f:
            color_map = json.load(f)
        logging.info("Color map loaded")
        return {int(k): np.array(v, dtype=np.float32) / 255.0 for k, v in color_map.items()}

    def read_bin(self, file_path: str) -> np.ndarray:
        """
        Reads a binary file and returns its contents as a numpy array.

        Parameters:
            file_path (str): Path to the binary file.

        Returns:
            np.ndarray: Numpy array containing the contents of the file.
        """
        return np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)

    def read_label(self, file_path: str) -> np.ndarray:
        """
        Reads a label file and returns its contents as a numpy array.

        Parameters:
            file_path (str): Path to the label file.

        Returns:
            np.ndarray: Numpy array containing the semantic labels.
        """
        label = np.fromfile(file_path, dtype=np.uint32)
        sem_label = label & 0xFFFF  # Semantic label in lower half
        return sem_label

    def read_poses(self, file_path: str) -> List[np.ndarray]:
        """
        Reads a poses file and returns a list of transformation matrices.

        Parameters:
            file_path (str): Path to the poses file.

        Returns:
            List[np.ndarray]: List of transformation matrices.
        """
        poses = []
        with open(file_path) as f:
            for line in f:
                pose = np.fromstring(line, dtype=float, sep=' ').reshape(3, 4)
                pose = np.vstack((pose, [0, 0, 0, 1]))
                poses.append(pose)
        return poses

    def apply_transform(self, points: np.ndarray, transform: np.ndarray) -> np.ndarray:
        """
        Applies a transformation to a set of points.

        Parameters:
            points (np.ndarray): Numpy array of points.
            transform (np.ndarray): Transformation matrix.

        Returns:
            np.ndarray: Transformed points.
        """
        hom_points = np.hstack((points, np.ones((points.shape[0], 1))))
        transformed_points = hom_points @ transform.T
        return transformed_points[:, :3]

    def update_point_cloud(self, vis: o3d.visualization.Visualizer, idx: int) -> None:
        """
        Updates the point cloud for visualization without ego-motion compensation.

        Parameters:
            vis (o3d.visualization.Visualizer): Open3D visualizer.
            idx (int): Index of the scan file to be visualized.
        """
        scan_file = os.path.join(self.scan_dir, self.scan_files[idx])
        label_file = os.path.join(self.label_dir, self.label_files[idx])
        
        points = self.read_bin(scan_file)[:, :3]
        labels = self.read_label(label_file)
        
        if points.shape[0] == 0:
            logging.warning(f"No points found in scan {scan_file}")
            return
        
        colors = np.array([self.color_map.get(label, [0.5, 0.5, 0.5]) for label in labels])
        
        self.pcd.points = o3d.utility.Vector3dVector(points)
        self.pcd.colors = o3d.utility.Vector3dVector(colors)
        
        if self.geometry_added:
            vis.remove_geometry(self.pcd, reset_bounding_box=False)
        
        vis.add_geometry(self.pcd, reset_bounding_box=not self.geometry_added)
        self.geometry_added = True
        
        vis.update_renderer()
        self.set_view_status(vis)
        logging.info(f"Frame {idx + 1}/{len(self.scan_files)} computed")

    def update_point_cloud_with_ego_motion(self, vis: o3d.visualization.Visualizer, idx: int) -> None:
        """
        Updates the point cloud for visualization with ego-motion compensation.

        Parameters:
            vis (o3d.visualization.Visualizer): Open3D visualizer.
            idx (int): Index of the scan file to be visualized.
        """
        scan_file = os.path.join(self.scan_dir, self.scan_files[idx])
        label_file = os.path.join(self.label_dir, self.label_files[idx])
        
        points = self.read_bin(scan_file)[:, :3]
        labels = self.read_label(label_file)
        
        if points.shape[0] == 0:
            logging.warning(f"No points found in scan {scan_file}")
            return
        
        poses = self.read_poses(self.pose_file)
        transformed_points = self.apply_transform(points, poses[idx])
        
        colors = np.array([self.color_map.get(label, [0.5, 0.5, 0.5]) for label in labels])
        
        self.pcd.points = o3d.utility.Vector3dVector(transformed_points)
        self.pcd.colors = o3d.utility.Vector3dVector(colors)
        
        if self.geometry_added:
            vis.remove_geometry(self.pcd, reset_bounding_box=False)
        
        vis.add_geometry(self.pcd, reset_bounding_box=not self.geometry_added)
        self.geometry_added = True
        
        vis.update_renderer()
        self.set_view_status(vis)
        logging.info(f"Frame {idx + 1}/{len(self.scan_files)} computed with ego-motion compensation")

    def set_view_status(self, vis: o3d.visualization.Visualizer) -> None:
        """
        Sets the view status of the visualizer.

        Parameters:
            vis (o3d.visualization.Visualizer): Open3D visualizer.
        """
        with open(self.view_status_file) as f:
            view_status = json.load(f)
        sio = StringIO()
        json.dump(view_status,sio)
        view_status_string = sio.getvalue()
        vis.set_view_status(view_status_string)
        logging.info("View status loaded")

    def visualize(self) -> None:
        """
        Visualizes the point cloud sequence without ego-motion compensation.
        """
        vis = o3d.visualization.VisualizerWithKeyCallback()
        vis.create_window(width=1920, height=1055)
        
        current_idx = [0]
        self.update_point_cloud(vis, current_idx[0])
        
        coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
        vis.add_geometry(coordinate_frame)

        def next(vis: o3d.visualization.Visualizer) -> None:
            if current_idx[0] < len(self.scan_files) - 1:
                current_idx[0] += 1
                self.update_point_cloud(vis, current_idx[0])
        
        def prev(vis: o3d.visualization.Visualizer) -> None:
            if current_idx[0] > 0:
                current_idx[0] -= 1
                self.update_point_cloud(vis, current_idx[0])
        
        def next20(vis: o3d.visualization.Visualizer):
            stepsize = 20
            if current_idx[0] < len(self.scan_files) -1 - stepsize:
                current_idx[0] += stepsize
                self.update_point_cloud(vis, current_idx[0])
       
        def prev20(vis: o3d.visualization.Visualizer):
            stepsize = 20
            if current_idx[0] < len(self.scan_files) -1 - stepsize:
                current_idx[0] -= stepsize
                self.update_point_cloud(vis, current_idx[0])
        
        vis.register_key_callback(262, next)
        vis.register_key_callback(263, prev)
        vis.register_key_callback(ord('N'),next20)
        vis.register_key_callback(ord('B'),prev20)
        
        vis.get_render_option().background_color = np.array([0, 0, 0])
        vis.get_render_option().point_size = self.point_size
        
        vis.poll_events()
        vis.update_renderer()
        self.set_view_status(vis)

        vis.run()
        vis.destroy_window()

    def visualize_with_ego_motion(self) -> None:
        """
        Visualizes the point cloud sequence with ego-motion compensation.
        """
        vis = o3d.visualization.VisualizerWithKeyCallback()
        vis.create_window(width=1920, height=1055)
        
        current_idx = [0]
        self.update_point_cloud_with_ego_motion(vis, current_idx[0])
        
        coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
        vis.add_geometry(coordinate_frame)

        def next(vis: o3d.visualization.Visualizer) -> None:
            stepsize = 1
            if current_idx[0] < len(self.scan_files) -1 - stepsize:
                current_idx[0] += stepsize
                self.update_point_cloud_with_ego_motion(vis, current_idx[0])
        
        def prev(vis: o3d.visualization.Visualizer) -> None:
            stepsize = 1
            if current_idx[0] >= stepsize :
                current_idx[0] -= stepsize
                self.update_point_cloud_with_ego_motion(vis, current_idx[0])
        

        def next20(vis: o3d.visualization.Visualizer):
            stepsize = 20
            if current_idx[0] < len(self.scan_files) -1 - stepsize:
                current_idx[0] += stepsize
                self.update_point_cloud_with_ego_motion(vis, current_idx[0])
       
        def prev20(vis: o3d.visualization.Visualizer):
            stepsize = 20
            if current_idx[0] < len(self.scan_files) -1 - stepsize:
                current_idx[0] -= stepsize
                self.update_point_cloud_with_ego_motion(vis, current_idx[0])
        
        vis.register_key_callback(262, next)
        vis.register_key_callback(263, prev)
        vis.register_key_callback(ord('N'),next20)
        vis.register_key_callback(ord('B'),prev20)
        
        vis.get_render_option().background_color = np.array([0, 0, 0])
        vis.get_render_option().point_size = self.point_size
        
        vis.poll_events()
        vis.update_renderer()
        self.set_view_status(vis)

        vis.run()
        vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


##### Setup

In [2]:
if __name__ == "__main__":
    base_dir = "./dataset/sequences/"
    seq_idx = "01"
    color_map_file = "./color_map.json"
    view_status_file = "./view_status.json"
    point_size = 3
    
    visualizer = SemanticKITTIVisualizer(base_dir, seq_idx, color_map_file, view_status_file, point_size)
    

2024-06-09 13:29:58,338 - INFO - Color map loaded
2024-06-09 13:29:58,341 - INFO - SemanticKITTIVisualizer initialized


#### Without Ego-motion

In [3]:
logging.info("Visualizing without ego-motion compensation")
visualizer.visualize()

2024-06-09 13:08:59,328 - INFO - Visualizing without ego-motion compensation
2024-06-09 13:09:00,248 - INFO - View status loaded
2024-06-09 13:09:00,248 - INFO - Frame 1/1101 computed
2024-06-09 13:09:00,273 - INFO - View status loaded
2024-06-09 13:09:13,676 - INFO - View status loaded
2024-06-09 13:09:13,677 - INFO - Frame 21/1101 computed
2024-06-09 13:09:14,531 - INFO - View status loaded
2024-06-09 13:09:14,532 - INFO - Frame 41/1101 computed
2024-06-09 13:09:16,236 - INFO - View status loaded
2024-06-09 13:09:16,237 - INFO - Frame 21/1101 computed
2024-06-09 13:09:17,161 - INFO - View status loaded
2024-06-09 13:09:17,162 - INFO - Frame 1/1101 computed
2024-06-09 13:09:18,225 - INFO - View status loaded
2024-06-09 13:09:18,226 - INFO - Frame 2/1101 computed
2024-06-09 13:09:19,161 - INFO - View status loaded
2024-06-09 13:09:19,162 - INFO - Frame 3/1101 computed
2024-06-09 13:09:19,890 - INFO - View status loaded
2024-06-09 13:09:19,891 - INFO - Frame 4/1101 computed
2024-06-09 1

#### With Ego-motion

In [3]:
logging.info("Visualizing with ego-motion compensation")
visualizer.visualize_with_ego_motion()

2024-06-09 13:30:01,583 - INFO - Visualizing with ego-motion compensation
2024-06-09 13:30:02,504 - INFO - View status loaded
2024-06-09 13:30:02,505 - INFO - Frame 1/1101 computed with ego-motion compensation
2024-06-09 13:30:02,530 - INFO - View status loaded
2024-06-09 13:30:06,249 - INFO - View status loaded
2024-06-09 13:30:06,250 - INFO - Frame 2/1101 computed with ego-motion compensation
2024-06-09 13:30:06,948 - INFO - View status loaded
2024-06-09 13:30:06,950 - INFO - Frame 3/1101 computed with ego-motion compensation
2024-06-09 13:30:07,586 - INFO - View status loaded
2024-06-09 13:30:07,587 - INFO - Frame 4/1101 computed with ego-motion compensation
2024-06-09 13:30:08,215 - INFO - View status loaded
2024-06-09 13:30:08,216 - INFO - Frame 5/1101 computed with ego-motion compensation
2024-06-09 13:30:08,898 - INFO - View status loaded
2024-06-09 13:30:08,899 - INFO - Frame 6/1101 computed with ego-motion compensation
2024-06-09 13:30:09,510 - INFO - View status loaded
2024-0